<a href="https://colab.research.google.com/github/escaduto/FireSpread/blob/master/GriddedLayers_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract daily weather variables based on unique ID 


## Setup

### Connect to Drive

In [1]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [2]:
import os
rootPath = "/content/drive/My Drive/California_FireTrends"
os.chdir(rootPath)

### Connect to GEE API

In [ ]:
# initialize and connect to GEE 
from google.colab import auth
auth.authenticate_user()
!earthengine authenticate
import ee 
ee.Initialize()

In [4]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

geemap package not installed. Installing ...


### Connect to Cloud Bucket

In [ ]:
# Connect to google cloud 
! gcloud auth login

### Load Libraries 

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [6]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import rtree

In [7]:
def listFiles_ByExt(rootPath, ext):
  '''
  retrieve file path + names based on extension
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
      for names in files: 
          if names.endswith(ext):
              file_list.append(os.path.join(path,names))
  return file_list

def createFolder(rootPath, folderName): 
  '''
  Create new folder in root path 
  '''
  folderPath = os.path.join(rootPath, folderName) 
  if not os.path.exists(folderPath):
      os.makedirs(folderPath)
  return folderPath 


def listFiles_ALL(rootPath):
  '''
  retrieve file path + names 
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
    for names in files: 
      file_list.append(os.path.join(path,names))
  return file_list

## Prepare Fire Perimeters

### Combine all interpolated surfaces into one file

In [5]:
rootPath = os.path.join(r'Products', 'By_Fire')
inter_files = listFiles_ByExt(rootPath,'.geojson')

In [6]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')

gdf = pd.concat([
    gpd.read_file(shp)
    for shp in inter_files
    ]).pipe(gpd.GeoDataFrame)

# gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
# gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

### Clip to CA Extent

Currently the study area is a bit outside state lines 

In [ ]:
gdf.crs

In [8]:
# check; set CRS 
state_path = '/content/drive/My Drive/COVID_FireTrends/Data/US_states/'
states = gpd.read_file(state_path)
CA_states = states[(states.STATE_NAME == 'California')]
CA_states = CA_states.to_crs('EPSG:3310')
CA_states_buff = CA_states
CA_states_buff['geometry'] = CA_states.geometry.buffer(5000)

In [49]:
CA_states_buff.to_file('Data/CA_Extent/California_5kmbuff.shp')

In [9]:
## clip to california extent 
Interpolated_CA = gpd.overlay(gdf, CA_states_buff, how='intersection')

In [ ]:
# Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
# Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

### Create unique ID column (numeric)

{FIREID}{YR}{JD}

In [ ]:
Interpolated_CA.sort_values(by=['Year', 'Fire'])
Interpolated_CA['FireID'] = 0
mask = Interpolated_CA.groupby(['Fire','Year'])['FireID'].transform(lambda x : len(x)>1)
Interpolated_CA.loc[mask,'FireID'] = Interpolated_CA.loc[mask,['Fire','Year']].astype(str).sum(1).factorize()[0] + 1

In [40]:
Interpolated_CA['Unique_ID'] = Interpolated_CA['FireID'].astype('str').str.zfill(3) + Interpolated_CA['YRJD'].astype('str').str[2:]
Interpolated_CA['Unique_ID'] = Interpolated_CA['Unique_ID'].astype('int64')

In [ ]:
Interpolated_CA.dtypes

In [ ]:
Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

### Export to Bucket and to Asset

In [ ]:
! gsutil -m cp -R 'Data/CA_Extent' gs://daily_fire_surfaces

In [ ]:
! earthengine upload table --asset_id=users/escaduto/CA_FireTrends/CA_Extent gs://daily_fire_surfaces/CA_Extent/California_5kmbuff.shp

In [ ]:
! gsutil -m cp -R 'Products/ALL_2012_2020' gs://daily_fire_surfaces

In [ ]:
! earthengine upload table --asset_id=users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020 gs://daily_fire_surfaces/ALL_2012_2020/ALL_2012_2020.shp

In [8]:
# import feature collection asset 
fire_perimeters = ee.FeatureCollection('users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020')
CA_Extent = ee.FeatureCollection('users/escaduto/CA_FireTrends/CA_Extent')

In [9]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')
Interpolated_CA = gpd.read_file(os.path.join(outpath, f'ALL_2012_2020.geojson'))

In [10]:
Interpolated_CA.columns

Index(['JulianDay', 'YRJD', 'Fire', 'Year', 'Month', 'Day', 'Date',
       'Area (acres)', 'Area (ha)', 'Time_Min', 'Time_Max', 'STATE_NAME',
       'DRAWSEQ', 'STATE_FIPS', 'SUB_REGION', 'STATE_ABBR', 'FireID',
       'Unique_ID', 'geometry'],
      dtype='object')

## Functions to GET Layers from GEE

#### Cloud Masks

In [18]:
def maskL8sr(image):  
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    # Get the pixel QA band.
    qa = image.select('pixel_qa');
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) and (qa.bitwiseAnd(cloudsBitMask).eq(0));
    return image.updateMask(mask);

Landsat 7 Mask

In [20]:
# cloud m
def cloudMaskL457(image):
  cloudShadowBitMask = (1 << 3);
  cloudsBitMask = (1 << 5);
  cloudsconfidenceBitMask = (1 << 7);
  qa = image.select('pixel_qa');
  mask = qa.bitwiseAnd(cloudsconfidenceBitMask).eq(0) and (qa.bitwiseAnd(cloudsBitMask).eq(0)) or (qa.bitwiseAnd(cloudShadowBitMask).eq(0));
  return image.updateMask(mask);


### A. Vegetation Indices (30m) 

1) MONTHLY NDVI, NDWI, NDMI

2) NDMI DELTA

3) MONTHLY EVI

4) ANNUAL NDVI, NDWI, NDMI

5) ANNUAL EVI




Landsat 8 Mask

#### [1] MONTHLY NDVI, NDWI, NDMI

In [201]:
from datetime import datetime, timedelta

def getStartEndDates(yr, mth):
  date_time_obj = datetime.strptime(str(yr) + "-" + str(mth), '%Y-%m') # create datetime object
  d = date_time_obj - timedelta(days=60) # subtract 2 months
  newyr = str(d.year)
  newmth = str(d.month)
  startDate = newyr + "-" + newmth + "-01"
  endDate = str(yr) + "-" + str(mth) + "-01"
  return startDate, endDate

In [202]:
def getMonthlyLandsat(yr, mth, LandsatCollection, Cloudmask):
  startDate, endDate = getStartEndDates(yr, mth)

  LandsatImage = ee.Image(ee.ImageCollection(LandsatCollection)
                  .filterDate(startDate, endDate)    
                  .sort('CLOUD_COVER')
                  .filterBounds(CA_Extent)
                  .map(Cloudmask)
                  .median());

  LandsatClip = LandsatImage.clip(CA_Extent)
  return LandsatClip 


def computeMonthlyVegIndices(yr, mth, LandsatCollection, Cloudmask, EVILayerName):
  Landsat = getMonthlyLandsat(yr, mth, LandsatCollection, Cloudmask)
  
  red = Landsat.select('B4')
  nir = Landsat.select('B5')
  swir = Landsat.select('B6')

  NDVI = nir.subtract(red).divide(nir.add(red)).rename('NDVI'); # nir-red 
  NDMI = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI'); # nir-swir
  NDMI_Delta = computeNDMIDelta(NDMI, mth)
  EVI = getMonthlylEVI(yr, mth, EVILayerName)

  Landsat = Landsat.addBands(NDMI).addBands(EVI).addBands(NDVI).addBands(NDMI_Delta)
  Landsat = Landsat.select(['NDVI', 'NDMI', 'EVI', 'NDMIDelta'])
  return (Landsat)

#### [2] NDMI DELTA

In [203]:
def computeNDMIDelta(NDMI, mth):
  baseLine = ee.Image(ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
                  .filter(ee.Filter.calendarRange(2012 - 5, 2012,'year'))
                  .filter(ee.Filter.calendarRange(int(mth),int(mth),'month'))
                  .map(cloudMaskL457)
                  .filterBounds(CA_Extent)
                  .median());                                                                                         
                                                                                            
  baseLineClip = baseLine.clip(CA_Extent)

  swir = baseLineClip.select('B5');
  nir = baseLineClip.select('B4');
  baseLineNDMI = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI_Base');  
  NDMIDelta = NDMI.select('NDMI').subtract(baseLineNDMI.select('NDMI_Base')).rename('NDMIDelta')      
  return NDMIDelta

#### [3] MONTHLY EVI

In [204]:
def getMonthlylEVI(yr, mth, EVILayerName):   #'LANDSAT/LC08/C01/T1_32DAY_EVI'
  startDate, endDate = getStartEndDates(yr, mth)
  EVI_monthly_collection = ee.Image(ee.ImageCollection(EVILayerName)
                          .filterDate(startDate, endDate)
                          .filterBounds(CA_Extent)
                          .select('EVI')
                          .median());
                                  
  EVI_Monthly = EVI_monthly_collection.clip(CA_Extent).rename('EVI');  
  return EVI_Monthly    

#### [4] ANNUAL NDVI, NDWI, NDMI

In [213]:
def getAnnualLandsat(yr, LandsatCollection, Cloudmask):
  start_yr = int(yr) - 1 
  Landsat_annual_Image = ee.Image(ee.ImageCollection(LandsatCollection)
                  .sort('CLOUD_COVER')
                  .filter(ee.Filter.date(f'{start_yr}-10-01', f'{yr}-04-30'))
                  .filterBounds(CA_Extent)
                  .map(Cloudmask)
                  .max());

  Landsat_Annual_Clip = Landsat_annual_Image.clip(CA_Extent)
  return Landsat_Annual_Clip

In [214]:
def computeAnnualVegIndices(yr, LandsatCollection, Cloudmask, EVILayerName):
  Landsat = getAnnualLandsat(yr, LandsatCollection, Cloudmask)

  red = Landsat.select('B4')
  nir = Landsat.select('B5')
  swir = Landsat.select('B6')

  NDVI = nir.subtract(red).divide(nir.add(red)).rename('Annual_NDVI');
  NDMI = nir.subtract(swir).divide(nir.add(swir)).rename('Annual_NDMI');
  EVI = getAnnualEVI(yr, EVILayerName)

  Landsat = Landsat.addBands(NDVI).addBands(NDMI).addBands(EVI)
  Landsat = Landsat.select(['Annual_NDVI', 'Annual_NDMI', 'Annual_EVI'])
  return (Landsat)

#### [5] ANNUAL EVI

In [215]:
def getAnnualEVI(yr, EVILayerName): #'LANDSAT/LC08/C01/T1_32DAY_EVI'
  start_yr = int(yr) - 1 
  EVI_annual_collection = ee.Image(ee.ImageCollection(EVILayerName)
                          .filter(ee.Filter.date(f'{start_yr}-10-01', f'{yr}-04-30'))
                          .filterBounds(CA_Extent)
                          .select('EVI')
                          .max());
                                  
  EVI_Annual = EVI_annual_collection.clip(CA_Extent).rename('Annual_EVI');   
  return EVI_Annual

### B. Daily Weather (resample to 30m) 

1) Gridmet 

2) Gridmet FFWI

3) Daymet 

4) Daymet VPD & RH


#### [1] GridMET

In [300]:
def getGRIDMET(yr, jd):
  gridmet_layers = ['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 
                    'tmmn','tmmx', 'vs', 'erc', 'bi', 'eto',
                    'fm100', 'fm1000', 'etr', 'vpd']

  gridmet_collection = ee.Image(ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
                            .filter(ee.Filter.calendarRange(int(yr), int(yr),'year'))
                            .filter(ee.Filter.calendarRange(int(jd),int(jd),'DAY_OF_YEAR'))
                            .select(gridmet_layers)
                            .mosaic())

  gridmet = gridmet_collection.clip(CA_Extent) 

  gridmet = getFFWI(gridmet)
  return gridmet

#### [2] FFWI

In [209]:
def getFFWI(gridmet):
  FFWI = gridmet.expression(
          "(b('rmax') < 10) ? (1 - (2 *( (0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) / (30 * 1.0))) + ((1.5 * ( (0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) /(30*1.0))**2)) - (0.5 * ((0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": (b('rmax') > 10 && b('rmax') <= 50) ? (1 - (2 *( (2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) / (30 * 1.0))) + ((1.5 * ( (2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) /(30*1.0))**2)) - (0.5 * ((2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": (b('rmax') > 50) ? (1 - (2 *((21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) / (30 * 1.0))) + ((1.5 * ( (21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) /(30*1.0))**2)) - (0.5 * ((21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": 0").rename('FFWI')

  gridmet_full = gridmet.addBands(FFWI)
  return gridmet_full

#### [3] DayMET

In [57]:
def getDAYMET(yr, jd):
  daymet_layers = ['tmax', 'srad', 'vp', 'tmin', 'prcp']

  daymet_collection = ee.Image(ee.ImageCollection('NASA/ORNL/DAYMET_V3')
                            .filter(ee.Filter.calendarRange(int(yr), int(yr),'year'))
                            .filter(ee.Filter.calendarRange(int(jd),int(jd),'DAY_OF_YEAR'))
                            .select(daymet_layers)
                            .mosaic())

  daymet = daymet_collection.clip(CA_Extent)

  daymet = getVPD_RH(daymet)

  return daymet 

#### [4] VPD & RH

In [56]:
def getVPD_RH(daymet):
  VP_amb = daymet.expression("610.78**((17.269 * tmin) / (237.3 + tmin))", {
    'tmin': daymet.select('tmin')
  })
    
  VP_sat = daymet.expression("610.78**((17.269 * (0.394 * tmin + 0.606 * tmax)) / (237.3 + (0.394 * tmin + 0.606 * tmax)))", {
    'tmin': daymet.select('tmin'),
    'tmax': daymet.select('tmax')
  })

  VP_Deficit = VP_sat.subtract(VP_amb).divide(1000).rename('DayMET_VPD')
  Rel_Humidity = VP_amb.divide(VP_sat).multiply(100).rename('DayMET_RH')

  daymet_full = daymet.addBands(VP_Deficit).addBands(Rel_Humidity);
  return daymet_full

### C. Topography (resample to 30m)

1) DEM, Slope, Aspect

2) Multiscale TPI 

3) Topo Diversity 

#### [1] DEM, Slope, Aspect

In [44]:
DEM_Image = ee.Image('USGS/NED').select('elevation');

DEM = DEM_Image.clip(CA_Extent).rename('DEM')
Slope = ee.Terrain.slope(DEM).rename('Slope')
Aspect = ee.Terrain.aspect(DEM).rename('Aspect')

topo = DEM.addBands(Slope).addBands(Aspect);

#### [2] TPI 

In [217]:
# 270 m (Valley (-) Ridge (+))
TPI_Image = ee.Image('CSP/ERGo/1_0/US/mTPI').select('elevation');

TPI = TPI_Image.clip(CA_Extent) 

# ridge > + 1 STDEV
# 2 upper slope > 0.5 STDV =< 1 STDV
# 3 middle slope> -0.5 STDV, < 0.5 STDV, slope > 5 deg
# 4 flats slope >= -0.5 STDV, =< 0.5 STDV , slope <= 5 deg
# 5 lower slopes >= -1.0 STDEV, < 0.5 STDV
# 6 valleys < -1.0 STDV

In [218]:
def getStats(operation, ras_Image, band_name):
  out = ras_Image.reduceRegion(**{
      'reducer': operation, 
      'bestEffort': True,
      'geometry': CA_Extent.geometry()
      });
  return out.getNumber(f'{band_name}').getInfo()


TPI_ST = getStats(ee.Reducer.stdDev(), TPI, 'elevation')
TPI_mean = getStats(ee.Reducer.mean(), TPI, 'elevation')
TPI_max = getStats(ee.Reducer.max(), TPI, 'elevation')
TPI_min = getStats(ee.Reducer.min(), TPI, 'elevation')

In [234]:
thresholds = ee.Image([TPI_min, TPI_mean - TPI_ST, 
                       TPI_mean - TPI_ST/2, TPI_mean + TPI_ST/2, TPI_mean + TPI_ST, TPI_max]);

TPI_Zones = TPI.expression(f'b(0) <= {TPI_mean - (2*(TPI_ST))} ? 1 : \
                             b(0) < {TPI_mean - (1/2*(TPI_ST))} ? 2 : \
                             b(0) < {TPI_mean + (1/2*(TPI_ST))} ? 3 : \
                             b(0) < {TPI_mean + (1*(TPI_ST))} ? 4 : \
                             b(0) < {TPI_max} ? 5 : 0').rename('TPI')

ridge = TPI_Zones.select(['TPI']).eq(1)
valley = TPI_Zones.select(['TPI']).eq(5)

####[3] Topo Diversity

In [45]:
topo_diversity_Image = ee.Image('CSP/ERGo/1_0/Global/ALOS_topoDiversity').select('constant');

topo_diversity= topo_diversity_Image.clip(CA_Extent).rename('TOPO_DIV')
topo = topo.addBands(topo_diversity)

### D. LandFire (30m)

USE: ee.Reducer.countDistinct()

1) Vegetation Type 

2) Vegetation Height 

#### [1] Vegetation Type

In [98]:
EVT_Image = ee.Image(ee.ImageCollection('LANDFIRE/Vegetation/EVT/v1_4_0').select('EVT').mosaic())

EVT = EVT_Image.clip(CA_Extent).rename('EVT')

#### [2] Vegetation Height

In [99]:
EVH_Image = ee.Image(ee.ImageCollection('LANDFIRE/Vegetation/EVH/v1_4_0').select('EVH').mosaic())

EVH = EVH_Image.clip(CA_Extent).rename('EVH')

In [100]:
Veg_TPI = TPI_Zones.addBands(EVH).addBands(EVT);

### E. Cost Distance to human settlement (resample to 30m)

In [280]:
roads = ee.FeatureCollection('TIGER/2016/Roads');

# // Create a source image (palm oil mill=1, others=0)
sources = ee.Image().toByte().paint(roads, 1);

# // Mask the sources image with itself.
sources = sources.updateMask(sources);
sources = sources.clip(CA_Extent)

# // The cost data is generated from slope 
cover = Slope.select(0);

# // Variable cost
cost = cover.lte(10) and (cover.gt(0)).multiply(1).add(
       cover.lte(30) and (cover.gt(10)).multiply(2)).add(
       cover.lte(50) and (cover.gt(30)).multiply(3)).add(
       cover.lte(60) and (cover.gt(50)).multiply(4)).add(
       cover.lte(100) and (cover.gt(60)).multiply(5))

# // Compute the cumulative cost to traverse the land cover.
cumulativeCost = cost.cumulativeCost(**{
  'source': sources,
  'maxDistance': 10 * 1000 
}).rename('travelCost')


## For each instance of fire, GET layers, Mosaic, Reduce, Save

Make sure all layers are aligned, with correct CRS projection and resolution

In [47]:
# import feature collection asset 
fire_perimeters = ee.FeatureCollection('users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020')
CA_Extent = ee.FeatureCollection('users/escaduto/CA_FireTrends/CA_Extent')

In [50]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')
Interpolated_CA = gpd.read_file(os.path.join(outpath, f'ALL_2012_2020.geojson'))

In [ ]:
Interpolated_CA.columns

In [48]:
def exportGEE(features, var, yrjd):
  exporttask = ee.batch.Export.table.toDrive(**{
  'collection': features,
  'folder': "Google EE results",
  'fileNamePrefix': f'{yrjd}_{var}',
  # 'selectors':(["Aspect","DEM","Slope"]),
  });

  exporttask.start()
  print(f'exporting {yrjd}_{var}.csv')


In [311]:
def GEEReducer(IMG, collection, reducer, scale):
  reduced = IMG.reduceRegions(**{
    'collection': collection,
    'reducer': reducer,
    'scale': scale,
  })
  return reduced

In [325]:
# get list of unique yrjd instances 
# get month year, JD, etc. 
L8_fires = Interpolated_CA[Interpolated_CA['Year'] >= 2013]
yr_jd_list  = L8_fires['YRJD'].unique().tolist()
yr_jd_list.sort(reverse=True)

In [ ]:
# 'LANDSAT/LE07/C01/T1_SR'
# 'LANDSAT/LC08/C01/T1_SR'

for yrjd in yr_jd_list:
  yr_jd_dt = datetime.strptime(yrjd, '%Y%j') 
  year = yr_jd_dt.year
  dy = yr_jd_dt.day
  mnth = yr_jd_dt.month
  mnth = str(mnth).zfill(2)
  jd = yrjd[-3:]
  print(jd, mnth, year)

  day_perimeter = fire_perimeters.filter(ee.Filter.eq('YRJD', yrjd))
  
  # reduce Image Collections 
  # Topography
  # Topo_Features = GEEReducer(topo, day_perimeter, ee.Reducer.mean(), 30)

  # Weather 
  daymet = getDAYMET(year, jd)
  # daymet_Features = GEEReducer(daymet, day_perimeter, ee.Reducer.mean(), 30)

  gridmet = getGRIDMET(year, jd)
  # gridmet_Features = GEEReducer(gridmet, day_perimeter, ee.Reducer.mean(), 30)

  # Vegetation 
  monthly_indices = computeMonthlyVegIndices(year, mnth, LandsatCollection = 'LANDSAT/LC08/C01/T1_SR', Cloudmask = maskL8sr, EVILayerName='LANDSAT/LC08/C01/T1_32DAY_EVI')
  # monthlyVeg_Features = GEEReducer(monthly_indices, day_perimeter, ee.Reducer.mean(), 30)
  annual_indices = computeAnnualVegIndices(year, LandsatCollection = 'LANDSAT/LC08/C01/T1_SR', Cloudmask = maskL8sr, EVILayerName='LANDSAT/LC08/C01/T1_32DAY_EVI')
  # annualVeg_Features = GEEReducer(annual_indices, day_perimeter, ee.Reducer.mean(), 30)

   # Human Travel Cost 
  # travelCost_Features = GEEReducer(cumulativeCost, day_perimeter, ee.Reducer.mean(), 30)

  allLayers = monthly_indices.addBands(annual_indices).addBands(gridmet).addBands(topo).addBands(cumulativeCost)
  all_Features = GEEReducer(allLayers, day_perimeter, ee.Reducer.mean(), 30)

  
  #[TODO] Fuel Type, Height, TPI, Fuel Model 
  # LandFire + TPI (Percent by unique value)
  #Veg_TPI_Features = GEEReducer(TPI_Zones, day_perimeter, ee.Reducer.countDistinct(), 30)
  #ee.Reducer.group(**{'reducer': ee.Reducer.count(),'groupField':0,'groupName':'TPI'})
  
  exportGEE(all_Features, 'dailyfire_zonalstats', yrjd)
  # exportGEE(Veg_TPI_Features, 'Veg_TPI', yrjd)
  # exportGEE(daymet_Features, 'DAYMET', yrjd)



In [109]:
os.getcwd()

'/content/drive/My Drive/California_FireTrends'

## Visualize Layers

In [283]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
};

ndviParams ={
  'min': -1,
  'max': 1,
  'palette': ['ffff00', '330033','7fffd4']
};

Map = emap.Map(center=[38.50178453635526,-122.74843617724784], zoom=10)
# Map.addLayer(TPI_Zones, {'palette': ['ffff00', '330033','7fffd4', 
#                                                          'b99879','cc8e7f', '268b07']}, 'TPI_class')
Map.addLayer(landsat, visParams , 'landsat')
# Map.addLayer(valley, {'min':0, 'max':1, 'palette': ['ffff00', '330033']} , 'valley')
# Map.addLayer(ridge, {'min':0, 'max':1, 'palette': ['ffff00', '330033']} , 'ridge')
Map.addLayer(cumulativeCost, {'min': 0, 'max': 5e3, 'palette': ['34a853', 'fbc034', 'ea4335', '4285f4']} , 'cumulativeCost')
Map.addLayerControl()
Map